In [1]:
from helper_functions import *
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(N)

#### Data preprocessing

In [3]:
train_input = 0.9*train_input
test_input = 0.9*test_input

train_input, test_input = normalize(train_input, test_input)

### Convolutional Neuronal Network

#### Predict digit target ∈ {0,1}

#### Define model

In [4]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        nb_hidden = 50
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=2)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.drop1 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64, nb_hidden)
        self.drop2 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(nb_hidden, 2)
        self.fc3 = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)),kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)),kernel_size=2))
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = self.drop1(x)
        x = F.relu(self.fc1(x.view(-1, 64)))
        x = self.drop2(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    
def train_model(model, train_input, train_target, mini_batch_size):
    eta = 1e-1
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=eta)
   
    for e in range(25):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss = sum_loss + loss.item()

def compute_nb_errors(model, input_, target, mini_batch_size):
    nb_errors = 0
    for b in range(0, input_.size(0), mini_batch_size):
        _,output = model(input_.narrow(0, b, mini_batch_size)).max(1)
        for k in range(mini_batch_size):
            if target[b + k] != output[k].item():
                nb_errors = nb_errors + 1
    return nb_errors

#### Train and test model

In [5]:
mini_batch_size = 100
model = Net2()

for k in range(15):
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('Final digit comparison error on test set: {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                  nb_test_errors, test_input.size(0)))

Final digit comparison error on test set: 23.30% 233/1000
Final digit comparison error on test set: 20.40% 204/1000
Final digit comparison error on test set: 19.70% 197/1000
Final digit comparison error on test set: 21.20% 212/1000
Final digit comparison error on test set: 20.20% 202/1000
Final digit comparison error on test set: 20.40% 204/1000
Final digit comparison error on test set: 19.60% 196/1000
Final digit comparison error on test set: 20.50% 205/1000
Final digit comparison error on test set: 19.50% 195/1000
Final digit comparison error on test set: 20.00% 200/1000
Final digit comparison error on test set: 20.00% 200/1000
Final digit comparison error on test set: 19.40% 194/1000
Final digit comparison error on test set: 20.40% 204/1000
Final digit comparison error on test set: 18.90% 189/1000
Final digit comparison error on test set: 20.80% 208/1000
